In [10]:
import matplotlib.pyplot as plt
import geopandas as gpd
import rasterio
from rasterio.plot import show
import numpy as np

from matplotlib.lines import Line2D
from matplotlib.patches import Patch

In [11]:
path_shp_cat = r'C:\Documents\OneDrive - Ingenio Azucarero Guabira S.A\INFORMACION GENERAL\CATASTRO\NUEVO\catastro_S06_SABADO.shp'
gdf_cat = gpd.read_file(path_shp_cat)
gdf_prop = gdf_cat[gdf_cat['unidad_01']==30]
gdf_prop = gdf_prop[['unidad_01', 'unidad_02', 'unidad_03', 'unidad_04', 'unidad_05', 'area', 'geometry']]

In [13]:
gdf_prop.head(3)

,unidad_01,unidad_02,unidad_03,unidad_04,unidad_05,area,geometry
70,30.0,CAMPO DULCE,41594.0,AGROPECUARIA CAMPO DULCE S.R.L.,EP-L26,44.606623,"POLYGON ((500070.322 8095568.798, 500053.889 8..."
71,30.0,CAMPO DULCE,41594.0,AGROPECUARIA CAMPO DULCE S.R.L.,EP-L2,22.961028,"POLYGON ((500165.099 8095280.914, 500742.525 8..."
72,30.0,CAMPO DULCE,41594.0,AGROPECUARIA CAMPO DULCE S.R.L.,EP-L30,27.634199,"POLYGON ((499189.807 8095282.481, 499247.624 8..."


In [18]:
def generar_plano_pulv(shp_prop, shp_recorrido, path_tif, path_out):
    # Cargar la imagen TIFF con rasterio
    with rasterio.open(path_tif) as src:
        extent = [src.bounds.left, src.bounds.right, src.bounds.bottom, src.bounds.top]  # Límites geoespaciales
        img = src.read([1, 2, 3])  # Leer las bandas RGB
    
    # Crear el plano
    fig, ax = plt.subplots(figsize=(15, 15))
    
    # Mostrar la imagen de fondo (TIFF) con coordenadas georreferenciadas
    show(img, transform=src.transform, ax=ax)
    
    #facecolor=(R, G, B, Alpha)
    shp_prop.plot(ax=ax, edgecolor='blue', facecolor=(0, 0, 0, 0), linewidth=1)
    shp_recorrido.plot(ax=ax, color='red', linewidth=0.5, linestyle="-", label="Líneas")

    # Agregar etiquetas con lote mas area
    for idx, row in shp_prop.iterrows():
        if row.geometry is not None:
            label = f"{row['unidad_05']}\n{round(row['area'], 2)} ha"
            centroid = row.geometry.representative_point()  # Mejor que .centroid en algunos casos
            ax.text(centroid.x, centroid.y, label, fontsize=7, ha='center', color='black', weight=1000,
                    bbox=dict(facecolor='white', edgecolor='none', alpha=0.2, pad=0))

    # Configurar título
    nom_prop = shp_prop["unidad_02"].iloc[0]
    ax.set_title(nom_prop, fontsize=14)

    # 🔍 **Ajustar los límites del mapa para hacer zoom en la propiedad**
    bounds = shp_prop.total_bounds  # [xmin, ymin, xmax, ymax]
    ax.set_xlim([bounds[0] - 400, bounds[2] + 400])
    ax.set_ylim([bounds[1] - 400, bounds[3] + 400])

    # 🔹 **Agregar Leyenda**
    legend_elements = [
        #facecolor=(R, G, B, Alpha)
        Patch(facecolor=(0, 0, 0, 0), edgecolor='blue', label='Lotes'),
        # Líneas (Caminos, canales, etc.)
        Line2D([0], [0], color='red', linewidth=1, linestyle="-", label='Recorrido de Dron'),
    ]
    ax.legend(handles=legend_elements, loc='upper right', fontsize=10, frameon=True, facecolor='white', edgecolor='black')

    # Guardar plano
    fig.savefig(path_plano, dpi = 300, bbox_inches='tight')
    
    # Mostrar el mapa
    #plt.show()

In [19]:
path_shp_recorrido = r"C:\vuelos\S09_AGROP_FLAMBOYAN"
shp_recorrido = gpd.read_file(path_shp_recorrido)

#path_tif = r"C:\Users\bismarksr\Downloads\Imagen_Real_RGB.tif"
path_tif = r"C:\Users\bismarksr\Downloads\SENTINEL_2025-02-12_UTM_2.tif"
path_plano = r"C:\Users\bismarksr\Downloads\plano_30.jpg"

prop = gdf_cat[gdf_cat['unidad_01']==29]
generar_plano_pulv(prop, shp_recorrido, path_tif, path_plano)